In [1]:
import os
import sys

import torch

# Append python path - needed to import text_recognizer
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/niall.turbitt/opt/anaconda3/envs/text-recognizer/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import text_recognizer.data
import text_recognizer.models

import training.run_experiment

In [ ]:

# Configure
emnist = text_recognizer.data.EMNIST()
print(emnist.__doc__)

In [ ]:
emnist.prepare_data()  # download, save to disk
emnist.setup()  # create torch.utils.data.Datasets, create train/val split


In [ ]:
x, y = next(iter(emnist.train_dataloader()))

In [ ]:
import random
import matplotlib.pyplot as plt


idx = random.randint(0, len(x) - 1)

print(emnist.mapping[y[idx]])
plt.imshow(x[idx].view(28,28))

In [ ]:
data_config = emnist.config()

print(data_config)

In [ ]:
cnn = text_recognizer.models.CNN(data_config)
print(cnn)

In [ ]:
[p.numel() for p in cnn.parameters()] 
# conv weight + bias, conv weight + bias, fc weight + bias, fc weight + bias

In [ ]:
[p.shape for p in cnn.parameters()]

In [ ]:
biggest_layer = [p for p in cnn.parameters() if p.numel() == max(p.numel() for p in cnn.parameters())][0]
biggest_layer.shape, cnn.fc_input_dim

In [2]:
%run ../training/run_experiment.py --model_class CNN --data_class EMNIST 

Missing logger folder: training/logs/lightning_logs
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type               | Params
------------------------------------------------------
0 | model          | CNN                | 1.7 M 
1 | model.conv1    | ConvBlock          | 640   
2 | model.conv2    | ConvBlock          | 36.9 K
3 | model.dropout  | Dropout            | 0     
4 | model.max_pool | MaxPool2d          | 0     
5 | model.fc1      | Linear             | 1.6 M 
6 | model.fc2      | Linear             | 10.7 K
7 | train_acc      | MulticlassAccuracy | 0     
8 | val_acc        | MulticlassAccuracy | 0     
9 | test_acc       | MulticlassAccuracy | 0     
-----------------------------

Epoch 0:   0%|          | 0/2542 [00:00<?, ?it/s]                          

In [ ]:
training.run_experiment.main()